In [5]:
from sklearn.datasets import load_files
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import pickle
import re
import konlpy
import nltk
import numpy as np

In [6]:
def clean_korean_documents(documents):
    #텍스트 정제 (HTML 태그 제거)
    for i, document in enumerate(documents):
        document = BeautifulSoup(document, 'html.parser').text 
        documents[i] = document

    #텍스트 정제 (특수기호 제거)
    for i, document in enumerate(documents):
        document = re.sub(r'[^ ㄱ-ㅣ가-힣]', '', document) #특수기호 제거, 정규 표현식
        documents[i] = document
        
    #텍스트 정제 (형태소 분석)
    for i, document in enumerate(documents):
        okt = konlpy.tag.Okt()
        clean_words = []
        for word in okt.pos(document, stem=True): #어간 추출
            if word[1] in ['Noun', 'Verb', 'Adjective']: #명사, 동사, 형용사
                clean_words.append(word[0])
        document = ' '.join(clean_words)
        documents[i] = document
        
    #텍스트 정제 (불용어 제거)
    df = pd.read_csv('https://raw.githubusercontent.com/cranberryai/todak_todak_python/master/machine_learning_text/clean_korean_documents/korean_stopwords.txt', header=None)
    df[0] = df[0].apply(lambda x: x.strip())
    stopwords = df[0].to_numpy()
    nltk.download('punkt')
    for i, document in enumerate(documents):
        clean_words = [] 
        for word in nltk.tokenize.word_tokenize(document): 
            if word not in stopwords: #불용어 제거
                clean_words.append(word)
        documents[i] = ' '.join(clean_words)  
        
    return documents

##########데이터 로드

naver_news = load_files('D:/webservice/final/whatthispills/ml/newsData/', shuffle=True)

labels = ['정치', '경제', '사회', '생활/문화', '세계', '기술/IT', '연예', '스포츠']

##########데이터 분석

##########데이터 전처리

x_data = naver_news.data
x_data = x_data[:2] #데이터를 50개로 제한

x_data = [x.decode('utf-8') for x in x_data] #바이트를 문자열로 바꾸기
x_data = clean_korean_documents(x_data) #텍스트 정제

[nltk_data] Downloading package punkt to C:\Users\hi-
[nltk_data]     sinchon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
tfidf = TfidfVectorizer()
tfidf.fit(x_data)

TfidfVectorizer()

In [8]:
x_data[0:2]

['태양광 충전 되다 아이폰 테슬라 디넷 코리아 이정현 기자 도널드 트럼프 미국 대통령 푸틴 러시아 대통령 초상화 새기다 황금 아이폰 출시 하다 화제 모으다 러시아 업체 캐비아 이번 태양광 충전 가능하다 아이폰 를 판매 하다 매체 더버지 보도 하다 업체 주로 아이폰 애플 워치 제품 금 장식 개조 하다 고가 판매 하다 업체 아이폰 이름 아이폰 테슬라 제품 면 태양 전지 패널 있다 스마트폰 뒤지다 놓다 하다 충전 되어다 보조 배터리 형태 사용 하다 있다 업체 아이폰 직접 만들다 아니다 아이폰 탑재 되다 태양광 충전 기능 아이폰 직접 내장 되어다 있다 형태 아니다 아이폰 바깥 쪽 케이스 형태 탑재 되다 캐비아 동안 제공 하다 제품 마찬가지 고급스럽다 움 대화 하다 위해 노력 하다 아이폰 면 모서리 후 카메라 테두리 곳곳 금 장식 하다 제품 대다 한정 생산 되다 모델 아이폰 테슬라 가격 달러 약 만 모델 달러 약 만',
 '돈 꽃다발 돈 케이크 현금 선호 뚜렷하다 어버이날 선물 새 풍경 앵커 어버이날 오늘 부모님 어떻다 선물 하다 최근 지폐 만들다 꽃다발 인기 하다 현금 선호 하다 달라지다 선물 트렌드 장동 욱 기자 보도 하다 리포트 카네이션 정성 스레 포장 되다 꽃 휘감다 건 다르다 아니다 지폐 이다 카네이션 지폐 섞다 현금 꽃다발 플라워 박스 이다 현금 선물 하다 게 요즘 대세 이다 어버이날 선물 뭐 좋다 지다 휴대폰 사용자 지난 검색 데이터 만천 건 분석 하다 현금 로 위 이다 서지혜 꽃 가게 직원 어버이날 하다 늘다 꽃다발 플라워 박스 현금 꽃다발 주문 해주다 인터넷 현금 꽃다발 현금 케이크 만들다 방법 자세하다 소개 되다 있다 김인호 좋다 인천 시 학익동 성의 표현 의미 돈 꽃다발 주다 좋아하다 웃음 빵 터지다 자식 부모 휴대전화 메신저 상품권 선물 하다 늘 있다 주고받다 기도 편하다 양쪽 만족 이다 손 경연 김 나현 산시 남구 그냥 사다 마음 들다 생기다 있다 사고 싶다 필요하다 있다 유창 동국대 경영학 받다 쓸다 거 쓸다 거 를 해보다 하다 가장 용다 쓸

In [9]:
df = pd.DataFrame(tfidf.transform(x_data).toarray(), columns=tfidf.vocabulary_)
df.head()

,태양광,충전,되다,아이폰,테슬라,디넷,코리아,이정현,기자,도널드,...,쓸다,해보다,가장,용다,기발하다,자녀,담다,새롭다,떠오르다,조선
0,0.000000,0.050803,0.050803,0.000000,0.050803,0.000000,0.000000,0.000000,0.050803,0.050803,...,0.050803,0.000000,0.000000,0.000000,0.152409,0.050803,0.050803,0.000000,0.000000,0.000000
1,0.046384,0.000000,0.000000,0.046384,0.000000,0.046384,0.046384,0.046384,0.000000,0.000000,...,0.000000,0.046384,0.092768,0.417455,0.000000,0.000000,0.000000,0.046384,0.046384,0.046384


In [10]:
df.iloc[0].sort_values(ascending=False)

지폐      0.609636
경영학     0.397614
카네이션    0.203212
자식      0.203212
주문      0.203212
          ...   
한정      0.000000
곳곳      0.000000
테두리     0.000000
카메라     0.000000
조선      0.000000
Name: 0, Length: 171, dtype: float64

In [11]:
df.iloc[1].sort_values(ascending=False)

용다      0.417455
뚜렷하다    0.371071
황금      0.324687
경영학     0.297023
박스      0.278303
          ...   
전지      0.000000
좋다      0.000000
대세      0.000000
요즘      0.000000
주로      0.000000
Name: 1, Length: 171, dtype: float64